---

# VODAFONE BIG DATA & AI TEAM  
## Modül 3: ADVANCED RAG (Retrieval-Augmented Generation) — COMPLETE EDITION  
### Lead AI Engineer Training — 2025  

---

Bu notebook, enterprise-grade RAG sistemlerinin en güncel **LangChain v1.x** ve  
**Azure OpenAI API** ile nasıl kurulacağını kapsamlı şekilde gösterir.

**📅 Güncellenme:** Ocak 2025  
**🔧 LangChain Version:** 1.2.7+ (Latest)  
**☁️ Azure OpenAI API Version:** 2024-12-01-preview  
**🔓 Open Source Reranker:** BGE v2-m3  

---

### Kapsam
- ✅ Basic RAG Implementation  
- ✅ Hybrid Search (BM25 + Semantic)  
- ✅ Advanced Reranking (BGE)  
- ✅ Multi-Query Retrieval  
- ✅ Parent Document Retrieval  
- ✅ Agent-based RAG  
---


# 📦 SECTION 1: KÜTÜPHANE KURULUMU (2025 - Latest Versions)

In [1]:
# ============================================================================
# 📦 SECTION 1: INSTALL (STABLE FIX - 2025/2026)
# ============================================================================

print("="*80)
print("📦 KÜTÜPHANELER YÜKLENİYOR (Dependency Fix)")
print("="*80)

# 1. Pip upgrade
!pip install -q --upgrade pip setuptools wheel

# 2. ÖNCE KRİTİK BAĞIMLILIKLARI SABİTLE (Çakışmayı önlemek için en başta çalışmalı)

!pip install -q "numpy==1.26.4" "pandas==2.2.3"
!pip install -q "pydantic>=2.0,<2.13.0"

# 3. Core LangChain (Modular)
!pip install -q --upgrade "langchain==1.2.7" "langchain-core==1.2.7"
!pip install -q --upgrade "langchain-community>=0.4.1"
!pip install -q --upgrade "langchain-openai>=1.1.7"
!pip install -q --upgrade "langchain-text-splitters>=1.1.0"
!pip install -q --upgrade "langchain-classic>=1.0.1"

# 4. OpenAI SDK
!pip install -q --upgrade "openai>=1.54.0"

# 5. Vector Stores & Document Processing
!pip install -q --upgrade "faiss-cpu>=1.8.0" "chromadb>=0.5.20"
!pip install -q --upgrade "pypdf>=4.3.0" "python-docx>=1.1.2" "unstructured>=0.16.0" "tiktoken>=0.8.0"

# 6. Advanced RAG & Graph
!pip install -q --upgrade "rank-bm25>=0.2.2" "sentence-transformers>=3.3.0" "FlagEmbedding"

# 7. Utilities
!pip install -q --upgrade "python-dotenv>=1.0.1"

# --- FIX: FlagEmbedding <-> transformers uyumu ---
!pip -q uninstall -y FlagEmbedding transformers tokenizers

# FlagEmbedding ekosisteminde yaygın uyumlu kombinasyon
!pip -q install "transformers==4.44.2" "tokenizers<0.21" "FlagEmbedding>=1.2.11"

# (opsiyonel) hız/uyumluluk için
!pip -q install -U accelerate

print("\n✅ Kurulum tamamlandı!")
print("⚠️ ÖNEMLİ: Eğer hala hata alıyorsanız 'Runtime > Restart Session' yapıp sadece import kısmını çalıştırın.")

📦 KÜTÜPHANELER YÜKLENİYOR (Dependency Fix)

✅ Kurulum tamamlandı!
⚠️ ÖNEMLİ: Eğer hala hata alıyorsanız 'Runtime > Restart Session' yapıp sadece import kısmını çalıştırın.


In [2]:
import pandas as pd

In [3]:
import numpy as np
import pandas as pd

print(np.__version__)
print(pd.__version__)


1.26.4
2.2.3


# 📚 SECTION 2: IMPORTS (2025 Best Practices)

In [4]:
# ============================================================================
# 📚 SECTION 2: IMPORTS (2025/2026 - Modular Best Practices)
# ============================================================================

print("\n" + "="*80)
print("📚 IMPORT İŞLEMLERİ")
print("="*80)

import os
import sys
import warnings
import json
import time
from typing import List, Dict, Optional, Any, Tuple, Union
from datetime import datetime
from pathlib import Path
from collections import defaultdict

warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

import numpy as np
import pandas as pd
import networkx as nx

from rank_bm25 import BM25Okapi
import tiktoken

# --- LangChain Core (LCEL + temel tipler) ---
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, BaseMessage

# Store (ParentDocumentRetriever / docstore vb.)
from langchain_core.stores import InMemoryStore  # core store :contentReference[oaicite:1]{index=1}

# --- Text Splitters (ayrı paket) ---
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    TokenTextSplitter,
    CharacterTextSplitter,
)

# --- Document Loaders (community) ---
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    DirectoryLoader,
    UnstructuredFileLoader,
    CSVLoader,
)

# --- Azure OpenAI (partner) ---
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

# --- Vector Stores (community) ---
from langchain_community.vectorstores import FAISS, Chroma
try:
    from langchain_community.vectorstores.utils import DistanceStrategy
except Exception:
    DistanceStrategy = None

# --- Retrievers (v1+’ta legacy/taşınanlar = langchain-classic) ---
from langchain_classic.retrievers import (
    EnsembleRetriever,
    MultiQueryRetriever,
    ParentDocumentRetriever,
    ContextualCompressionRetriever,
)

from langchain_classic.retrievers.document_compressors import (
    LLMChainExtractor,
    EmbeddingsFilter,
)

# BM25 Retriever (community)
from langchain_community.retrievers import BM25Retriever

# --- Chains & Memory (v1+’ta classic altında) ---
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain, LLMChain

# create_* helpers (bazı ortamlarda classic altında öneriliyor)
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.history_aware_retriever import create_history_aware_retriever

# --- Tools & Agents ---
from langchain_core.tools import tool, Tool, BaseTool, StructuredTool
from langchain_core.tools import tool, Tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.language_models.base import BaseLanguageModel
from langchain.agents import create_agent

# --- BGE Reranker (Open Source) ---
try:
    from FlagEmbedding import FlagReranker, FlagLLMReranker
    BGE_AVAILABLE = True
    print("✅ BGE Reranker imported successfully!")
except Exception:
    BGE_AVAILABLE = False
    print("⚠️ BGE Reranker not available. Install with: pip install FlagEmbedding")

# --- Sentence Transformers (Fallback) ---
try:
    from sentence_transformers import CrossEncoder
    SENTENCE_TRANSFORMERS_AVAILABLE = True
except Exception:
    SENTENCE_TRANSFORMERS_AVAILABLE = False
    print("⚠️ Sentence Transformers not available")

print("\n✅ Import işlemleri tamamlandı!")
print(f"   🔓 BGE Reranker: {'Available ✅' if BGE_AVAILABLE else 'Not Available ❌'}")
print(f"   🤖 Sentence Transformers: {'Available ✅' if SENTENCE_TRANSFORMERS_AVAILABLE else 'Not Available ❌'}")
print(f"\n📌 Python Version: {sys.version.split()[0]}")



📚 IMPORT İŞLEMLERİ
✅ BGE Reranker imported successfully!

✅ Import işlemleri tamamlandı!
   🔓 BGE Reranker: Available ✅
   🤖 Sentence Transformers: Available ✅

📌 Python Version: 3.12.12


# 🔑 SECTION 3: AZURE OPENAI CONFIGURATION

In [ ]:
print("\n" + "="*80)
print("🔑 AZURE OPENAI CONFIGURATION")
print("="*80)

# Azure OpenAI Credentials (PRODUCTION - Use environment variables!)
AZURE_OPENAI_API_KEY = "API_KEY"
AZURE_OPENAI_ENDPOINT = "API_Endpoint"
AZURE_OPENAI_API_VERSION = "2024-12-01-preview"  # Latest API version

# Model Deployments (Your specific deployment names)
DEPLOYMENT_NAME_GPT = "vodafone_rag_module"  # GPT-4 deployment
DEPLOYMENT_NAME_EMBEDDING = "vodafone_rag_module_embeddings"  # Embedding deployment

# Set environment variables (Best practice for security)
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION

print("✅ Azure OpenAI konfigürasyonu tamamlandı!")
print(f"   🌐 Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"   📅 API Version: {AZURE_OPENAI_API_VERSION}")
print(f"   🤖 GPT Deployment: {DEPLOYMENT_NAME_GPT}")
print(f"   📊 Embedding Deployment: {DEPLOYMENT_NAME_EMBEDDING}")


🔑 AZURE OPENAI CONFIGURATION
✅ Azure OpenAI konfigürasyonu tamamlandı!
   🌐 Endpoint: https://sd-rg.cognitiveservices.azure.com/
   📅 API Version: 2024-12-01-preview
   🤖 GPT Deployment: vodafone_rag_module
   📊 Embedding Deployment: vodafone_rag_module_embeddings


# 🧪 SECTION 4: MODEL INITIALIZATION & TESTING

In [6]:
print("\n" + "="*80)
print("📊 Embedding Model Testi")
print("="*80)

# Azure OpenAI Embedding modelini initialize et
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=DEPLOYMENT_NAME_EMBEDDING,
    openai_api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

# Test metinleri
test_texts = [
    "Vodafone Türkiye'nin en büyük telekomünikasyon şirketlerinden biridir.",
    "Vodafone provides mobile and internet services to millions of customers.",
    "Yapay zeka ve büyük veri teknolojileri telekomünikasyon sektöründe devrim yaratıyor."
]

# Embeddings oluştur
print("\n🔄 Test metinleri için embedding'ler oluşturuluyor...")
test_embeddings = embeddings.embed_documents(test_texts)

# Sonuçları göster
print(f"\n✅ {len(test_embeddings)} metin için embedding oluşturuldu")
print(f"📏 Embedding boyutu: {len(test_embeddings[0])} dimensions")
print(f"📝 İlk embedding'in ilk 10 değeri: {test_embeddings[0][:10]}")

# Semantic similarity gösterimi
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

print("\n📊 Semantic Similarity Analizi:")
print(f"Metin 1 vs Metin 2: {cosine_similarity(test_embeddings[0], test_embeddings[1]):.4f}")
print(f"Metin 1 vs Metin 3: {cosine_similarity(test_embeddings[0], test_embeddings[2]):.4f}")
print(f"Metin 2 vs Metin 3: {cosine_similarity(test_embeddings[1], test_embeddings[2]):.4f}")


📊 Embedding Model Testi

🔄 Test metinleri için embedding'ler oluşturuluyor...

✅ 3 metin için embedding oluşturuldu
📏 Embedding boyutu: 3072 dimensions
📝 İlk embedding'in ilk 10 değeri: [-0.015259365551173687, 0.028038254007697105, 0.0004720104916486889, 0.03769439831376076, 0.026930898427963257, -0.006439275108277798, -0.02055252715945244, 0.006201193667948246, -0.008991730399429798, -0.022124972194433212]

📊 Semantic Similarity Analizi:
Metin 1 vs Metin 2: 0.5347
Metin 1 vs Metin 3: 0.4329
Metin 2 vs Metin 3: 0.2578


In [7]:
# Initialize Azure OpenAI Chat Model (Latest API)
print("\n2️⃣ Initializing Azure OpenAI Chat Model...")

llm = AzureChatOpenAI(
    azure_deployment=DEPLOYMENT_NAME_GPT,
    openai_api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    temperature=0,  # Deterministic responses
    max_tokens=2000,
    streaming=True,  # Enable streaming
    model_kwargs={
        "top_p": 0.95,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }
)

# Test LLM
print("   🔄 Testing LLM...")
test_messages = [
    SystemMessage(content="Sen Vodafone Türkiye'nin AI asistanısın."),
    HumanMessage(content="Merhaba! Kendini kısaca tanıt.")
]

test_response = llm.invoke(test_messages)

print(f"   ✅ Chat Model Ready!")
print(f"   🤖 Model: {DEPLOYMENT_NAME_GPT}")
print(f"   💬 Test Response: {test_response.content}...")

print("\n✅ Model initialization tamamlandı!")



2️⃣ Initializing Azure OpenAI Chat Model...
   🔄 Testing LLM...
   ✅ Chat Model Ready!
   🤖 Model: vodafone_rag_module
   💬 Test Response: Merhaba! Ben Vodafone Türkiye'nin yapay zeka asistanıyım. Size Vodafone ile ilgili her konuda yardımcı olmak için buradayım. Fatura sorgulama, paket bilgileri, kampanyalar, teknik destek ve daha fazlası hakkında sorularınızı yanıtlayabilirim. Size nasıl yardımcı olabilirim?...

✅ Model initialization tamamlandı!


# 📄 SECTION 5: VODAFONE SAMPLE DOCUMENT CREATION

In [8]:
print("\n" + "="*80)
print("📄 VODAFONE SAMPLE DOCUMENT CREATION")
print("="*80)

# Comprehensive Vodafone Product Catalog
vodafone_catalog_content = """
VODAFONE TÜRKİYE KAPSAMLI ÜRÜN VE HİZMET KATALOĞU 2024-2025
===========================================================

=== BİRİNCİ BÖLÜM: MOBİL PAKETLER ===

1. CONSUMER PAKETLERİ

1.1 Vodafone Genç Tarifesi Plus
Hedef Kitle: 18-28 yaş arası genç kullanıcılar
Aylık Ücret: 175 TL
Özellikler:
- 20 GB Genel İnternet Paketi
- Sosyal Medya Uygulamalarına Sınırsız İnternet (Instagram, TikTok, Twitter)
- Müzik Streaming Sınırsız (Spotify, Apple Music)
- Video Streaming 30 GB (YouTube, Netflix)
- Sınırsız Konuşma ve SMS
- 5G Desteği
- Uluslararası 100 Dakika
- Gaming Mode: Düşük ping garantisi
- Vodafone Points 2x kazanç

1.2 Vodafone Süper Paket Premium
Aylık Ücret: 350 TL
Özellikler:
- 75 GB 5G İnternet
- Sınırsız Konuşma ve SMS
- Avrupa Roaming Paketi Dahil (10 GB + 500 dakika)
- Vodafone TV Premium Paketi (3 ay ücretsiz)
- Cloud Storage 200 GB
- Cihaz Sigorta Paketi (%50 indirim)
- Öncelikli Müşteri Hizmetleri
- Airport Lounge Access (Yılda 4 kez)
- Vodafone Points 3x kazanç

1.3 Vodafone Aile Paketi
Aylık Ücret: 600 TL (4 hatta kadar)
Özellikler:
- Her hat için 30 GB İnternet (Toplam 120 GB paylaşımlı)
- Hatlar arası sınırsız konuşma
- Her hat sınırsız SMS
- Aile içi lokasyon paylaşımı
- Ebeveyn kontrol uygulaması
- Toplu fatura imkanı
- %20 cihaz alımlarında indirim

2. BUSINESS PAKETLERİ

2.1 Vodafone Business Starter
Aylık Ücret: 450 TL
Özellikler:
- 100 GB İnternet
- Sınırsız Konuşma ve SMS
- Kurumsal Güvenlik Paketi (VPN, MDM)
- Cloud Depolama 500 GB
- Öncelikli Destek Hattı
- Microsoft 365 Business Basic (3 ay deneme)
- Expense Management Tool

2.2 Vodafone Business Pro
Aylık Ücret: 750 TL
Özellikler:
- Sınırsız İnternet
- Sınırsız Konuşma ve SMS (Uluslararası Dahil)
- Advanced Security Suite (DLP, Threat Protection)
- Dedicated Account Manager
- SLA Garantisi %99.9 Uptime
- Cloud PBX Solution
- Video Conferencing Platform (25 katılımcı)
- Business Analytics Dashboard
- API Integration Support

=== İKİNCİ BÖLÜM: İNTERNET VE ALTYAPI ===

3. FIBER İNTERNET PAKETLERİ

3.1 Vodafone Fiber 100 Mbps Starter
Aylık Ücret: 400 TL
- Download: 100 Mbps
- Upload: 100 Mbps
- Wi-Fi 6 Modem (Ücretsiz)
- Sınırsız Kullanım
- 4 Cihaza Eşzamanlı Bağlantı Optimize
- Statik IP: +50 TL/ay

3.2 Vodafone Fiber 500 Mbps Gaming
Aylık Ücret: 600 TL
- Download/Upload: 500 Mbps
- Wi-Fi 6E Modem + Mesh Sistem
- Gaming QoS (Quality of Service)
- DDoS Protection
- Ping Optimization
- Port Forwarding Özelleştirilmiş
- 10 Cihaz Simultane Support

3.3 Vodafone Fiber 1000 Mbps Ultimate
Aylık Ücret: 850 TL
- Simetrik 1 Gbps
- Wi-Fi 7 Ready Modem + 2 Mesh Unit
- Öncelikli Network Routing
- Statik IP Dahil
- Advanced Parental Controls
- Network Analytics Dashboard
- 7/24 Premium Teknik Destek
- Kurulum Ücretsiz

4. KURUMSAL NETWORK ÇÖZÜMLER

4.1 MPLS Network
- Güvenli Site-to-Site Bağlantı
- SLA Garantili Bandwidth
- QoS Yönetimi
- 7/24 NOC Desteği
- Managed Router/Firewall
Fiyat: Özel Teklif (Bandwidth'e göre)

4.2 SD-WAN Solution
- Hybrid WAN Yönetimi
- Intelligent Path Selection
- Application Aware Routing
- Cloud Integration (Azure, AWS, GCP)
- Zero-Touch Provisioning
- Centralized Management Console
Fiyat: Lokasyon başına 1,500 TL/ay

=== ÜÇÜNCÜ BÖLÜM: DEĞER KATMALI HİZMETLER ===

5. VODAFONE TV+

5.1 Basic Paket - 75 TL/ay
- 60+ Canlı Kanal
- 7 Gün Geriye Dönük İzleme
- HD Yayın
- 2 Eşzamanlı Cihaz

5.2 Premium Paket - 150 TL/ay
- 120+ Canlı Kanal
- Spor Paketleri (Bein Sports, S Sport)
- 4K İçerik Desteği
- Netflix Basic Dahil
- Amazon Prime Video Dahil
- 5 Eşzamanlı Cihaz
- Sınırsız Kayıt

5.3 Ultimate Entertainment - 250 TL/ay
- Tüm Kanallar (150+)
- Tüm Spor Paketleri
- HBO Max, Disney+ Dahil
- Exclusive Vodafone Content
- 8 Eşzamanlı Cihaz

6. BULUT VE DEPOLAMA

6.1 Vodafone Cloud Personal
- 100 GB: Ücretsiz
- 500 GB: 30 TL/ay
- 2 TB: 75 TL/ay
- 5 TB: 150 TL/ay
Özellikler: Auto-backup, File Versioning, Collaboration

6.2 Vodafone Cloud Business
- 1 TB (5 kullanıcı): 200 TL/ay
- 5 TB (25 kullanıcı): 750 TL/ay
- 10 TB (Sınırsız): 1,500 TL/ay
Özellikler: Advanced Security, Compliance, Admin Console

7. GÜVENLİK HİZMETLERİ

7.1 Vodafone Secure
- Anti-Virus & Anti-Malware
- Safe Browsing
- Identity Protection
- Password Manager
- VPN Service (5 cihaz)
Fiyat: 40 TL/ay

7.2 Enterprise Security Suite
- Advanced Threat Protection
- SIEM (Security Information and Event Management)
- DLP (Data Loss Prevention)
- Email Security Gateway
- Vulnerability Assessment
- Penetration Testing (Yılda 2 kez)
Fiyat: 5,000 TL/ay (Lokasyon başına)

=== DÖRDÜNCÜ BÖLÜM: IoT VE DIJITAL DÖNÜŞÜM ===

8. IoT ÇÖZÜMLER

8.1 Akıllı Şehir Uygulamaları
- Smart Parking Sistemi
- Atık Yönetimi IoT Sensörleri
- Hava Kalitesi Monitörleme
- Akıllı Aydınlatma Kontrol
Fiyat: Proje Bazlı

8.2 Endüstriyel IoT (Industry 4.0)
- Predictive Maintenance Sensörleri
- Asset Tracking Solutions
- Environmental Monitoring
- Production Line Optimization
- Digital Twin Implementation
Fiyat: Cihaz başına 150 TL/ay + Platform Fee

8.3 Connected Car Solutions
- Gerçek Zamanlı Araç Takibi
- Sürüş Davranış Analizi
- Yakıt Tüketimi Optimizasyonu
- Bakım Uyarı Sistemi
- Fleet Management Dashboard
Fiyat: Araç başına 75 TL/ay

9. YAPAY ZEKA & ANALİTİK

9.1 AI-Powered Customer Analytics
- Customer Journey Mapping
- Churn Prediction
- Sentiment Analysis
- Personalization Engine
- A/B Testing Platform
Fiyat: 10,000 TL/ay (Kurulum: 50,000 TL)

9.2 Business Intelligence Suite
- Real-time Dashboards
- Predictive Analytics
- Custom Report Builder
- Data Warehouse Integration
- ML Model Training Support
Fiyat: Kullanıcı başına 500 TL/ay

=== BEŞİNCİ BÖLÜM: KAMPANYALAR VE AVANTAJLAR ===

10. AKTİF KAMPANYALAR (2024-2025)

10.1 Yeni Müşteri Hoşgeldin Kampanyası
- İlk 6 ay %40 indirim
- Modem bedava
- Kurulum ücretsiz
- İlk ay Vodafone TV Premium hediye
Geçerlilik: 31 Mart 2025

10.2 Sadakat Programı "Vodafone Stars"
Seviye Sistemi:
- Bronze: 0-1000 puan (1 yıl) - %0 indirim
- Silver: 1001-5000 puan (2 yıl) - %5 indirim
- Gold: 5001-15000 puan (3 yıl) - %10 indirim
- Platinum: 15000+ puan (5+ yıl) - %15 indirim + VIP Destek

10.3 Öğrenci Avantajları
- Vodafone Genç'e ek %15 indirim
- Campus Store'larda özel kampanyalar
- Eğitim içerikleri sınırsız
- Starbucks 10 TL hediye (her ay)

=== ALTINCI BÖLÜM: MÜŞTERİ HİZMETLERİ ===

11. DESTEK KANALLARI

11.1 Dijital Kanallar
- My Vodafone App (iOS, Android)
- Web: www.vodafone.com.tr
- WhatsApp: 0850 542 0 542
- Chatbot "Viki" (7/24)

11.2 Telefon Desteği
- Consumer: 542 (7/24)
- Business: 444 0 542 (7/24)

11.3 SLA (Service Level Agreements)
- Consumer: 48 saat
- Business Starter: 24 saat
- Business Pro: 4 saat (kritik)

=== YEDİNCİ BÖLÜM: TEKNİK ÖZELLİKLER ===

12. NETWORK ALTYAPI

12.1 5G Kapsama
- İstanbul: %95
- Ankara: %90
- İzmir: %85
- Diğer Şehirler: %70+

Güncellenme: 15 Ocak 2025
Versiyon: 4.0
"""

# Save document
doc_path = "vodafone_full_catalog.txt"
with open(doc_path, "w", encoding="utf-8") as f:
    f.write(vodafone_catalog_content)

print(f"✅ Vodafone Katalog Dokümanı Oluşturuldu")
print(f"   📄 Dosya: {doc_path}")
print(f"   📊 Toplam Karakter: {len(vodafone_catalog_content):,}")

# Calculate tokens
encoder = tiktoken.encoding_for_model("gpt-4")
tokens = encoder.encode(vodafone_catalog_content)
print(f"   🎯 Token Sayısı: {len(tokens):,}")
print(f"   💰 Embedding Cost Estimate: ~${len(tokens) * 0.00013 / 1000:.4f}")


📄 VODAFONE SAMPLE DOCUMENT CREATION
✅ Vodafone Katalog Dokümanı Oluşturuldu
   📄 Dosya: vodafone_full_catalog.txt
   📊 Toplam Karakter: 6,776
   🎯 Token Sayısı: 2,553
   💰 Embedding Cost Estimate: ~$0.0003


# 🔪 SECTION 6: ADVANCED TEXT SPLITTING & METADATA ENRICHMENT

In [9]:
print("\n" + "="*80)
print("🔪 ADVANCED TEXT SPLITTING & METADATA ENRICHMENT")
print("="*80)

# Load document
print("\n1️⃣ Loading document...")
loader = TextLoader(doc_path, encoding="utf-8")
documents = loader.load()
print(f"   ✅ Loaded {len(documents)} document(s)")

# Text Splitter Configuration
print("\n2️⃣ Configuring text splitter...")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""],
    is_separator_regex=False,
    keep_separator=True
)

# Split documents
splits = text_splitter.split_documents(documents)
print(f"   ✅ Created {len(splits)} chunks")
print(f"   📏 Average chunk size: {np.mean([len(s.page_content) for s in splits]):.0f} chars")

# Metadata Enrichment
print("\n3️⃣ Enriching metadata...")

def categorize_content(text: str) -> List[str]:
    """Categorize content based on keywords"""
    text_lower = text.lower()
    categories = []

    # Category mapping
    category_keywords = {
        'mobile': ['mobil', 'tarife', 'paket', 'genç', 'konuşma', 'sms', '5g'],
        'internet': ['fiber', 'internet', 'mbps', 'gbps', 'modem', 'wi-fi'],
        'tv': ['tv', 'kanal', 'yayın', 'netflix', 'prime'],
        'business': ['kurumsal', 'business', 'enterprise', 'şirket'],
        'iot': ['iot', 'akıllı', 'sensör', 'tracking', 'monitoring'],
        'security': ['güvenlik', 'security', 'vpn', 'firewall', 'dlp'],
        'cloud': ['bulut', 'cloud', 'depolama', 'storage'],
        'campaign': ['kampanya', 'indirim', 'promosyon', 'hediye'],
        'support': ['destek', 'müşteri', 'hizmet', 'support']
    }

    for category, keywords in category_keywords.items():
        if any(keyword in text_lower for keyword in keywords):
            categories.append(category)

    return categories if categories else ['general']

enriched_docs = []
for i, doc in enumerate(splits):
    # Categorize
    categories = categorize_content(doc.page_content)

    # Detect prices
    has_price = 'tl' in doc.page_content.lower() or '₺' in doc.page_content

    # Extract section (if available)
    lines = doc.page_content.split('\n')
    section = next((line.strip('= ') for line in lines if line.startswith('===')), 'Unknown')

    # Create enriched metadata
    metadata = {
        'chunk_id': i,
        'categories': categories,
        'has_price': has_price,
        'section': section,
        'char_count': len(doc.page_content),
        'source': 'vodafone_catalog',
        'version': '4.0',
        'date': '2025-01-15'
    }

    enriched_doc = Document(
        page_content=doc.page_content,
        metadata=metadata
    )
    enriched_docs.append(enriched_doc)

print(f"   ✅ Enriched {len(enriched_docs)} documents with metadata")

# Show example metadata
print(f"\n   📋 Example Metadata (Chunk #{7}):")
print(f"   {json.dumps(enriched_docs[7].metadata, indent=6, ensure_ascii=False)}")

# Category distribution
from collections import Counter
all_categories = [cat for doc in enriched_docs for cat in doc.metadata['categories']]
category_dist = Counter(all_categories)

print(f"\n   📊 Category Distribution:")
for cat, count in category_dist.most_common():
    print(f"      {cat}: {count} chunks")



🔪 ADVANCED TEXT SPLITTING & METADATA ENRICHMENT

1️⃣ Loading document...
   ✅ Loaded 1 document(s)

2️⃣ Configuring text splitter...
   ✅ Created 8 chunks
   📏 Average chunk size: 918 chars

3️⃣ Enriching metadata...
   ✅ Enriched 8 documents with metadata

   📋 Example Metadata (Chunk #7):
   {
      "chunk_id": 7,
      "categories": [
            "mobile",
            "tv",
            "business",
            "campaign",
            "support"
      ],
      "has_price": true,
      "section": "ALTINCI BÖLÜM: MÜŞTERİ HİZMETLERİ",
      "char_count": 735,
      "source": "vodafone_catalog",
      "version": "4.0",
      "date": "2025-01-15"
}

   📊 Category Distribution:
      mobile: 6 chunks
      tv: 5 chunks
      support: 5 chunks
      business: 5 chunks
      cloud: 4 chunks
      campaign: 4 chunks
      security: 4 chunks
      internet: 2 chunks
      iot: 1 chunks


# 🗄️ SECTION 7: VECTOR STORE CREATION (FAISS)

In [10]:
print("\n" + "="*80)
print("🗄️ VECTOR STORE CREATION")
print("="*80)

# FAISS Vector Store
print("\n1️⃣ Creating FAISS Vector Store...")
start_time = time.time()

faiss_vectorstore = FAISS.from_documents(
    documents=enriched_docs,
    embedding=embeddings,
    distance_strategy=DistanceStrategy.COSINE
)

faiss_time = time.time() - start_time
print(f"   ✅ FAISS Vector Store Created")
print(f"   📊 Indexed Vectors: {faiss_vectorstore.index.ntotal}")
print(f"   ⏱️  Creation Time: {faiss_time:.2f}s")

# Save FAISS index
faiss_path = "./vodafone_faiss_index"
faiss_vectorstore.save_local(faiss_path)
print(f"   💾 Saved to: {faiss_path}")

# Test similarity search
print("\n3️⃣ Testing Similarity Search...")
test_query = "5G destekli paketler ve fiyatları"

print(f"   🔍 Query: '{test_query}'")
print(f"\n   📊 FAISS Results (Top 3):")
faiss_results = faiss_vectorstore.similarity_search_with_score(test_query, k=3)

for i, (doc, score) in enumerate(faiss_results, 1):
    print(f"\n      {i}. Distance: {score:.4f}")
    print(f"         Categories: {doc.metadata.get('categories', [])}")
    print(f"         Content: {doc.page_content[:120]}...")

print("\n✅ Vector stores ready for retrieval!")



🗄️ VECTOR STORE CREATION

1️⃣ Creating FAISS Vector Store...
   ✅ FAISS Vector Store Created
   📊 Indexed Vectors: 8
   ⏱️  Creation Time: 0.89s
   💾 Saved to: ./vodafone_faiss_index

3️⃣ Testing Similarity Search...
   🔍 Query: '5G destekli paketler ve fiyatları'

   📊 FAISS Results (Top 3):

      1. Distance: 0.7569
         Categories: ['mobile', 'tv', 'cloud', 'campaign', 'support']
         Content: VODAFONE TÜRKİYE KAPSAMLI ÜRÜN VE HİZMET KATALOĞU 2024-2025
...

      2. Distance: 0.7832
         Categories: ['mobile', 'business', 'security', 'cloud', 'campaign', 'support']
         Content: 1.3 Vodafone Aile Paketi
Aylık Ücret: 600 TL (4 hatta kadar)
Özellikler:
- Her hat için 30 GB İnternet (Toplam 120 GB pa...

      3. Distance: 0.8172
         Categories: ['mobile', 'internet', 'business', 'security', 'support']
         Content: === İKİNCİ BÖLÜM: İNTERNET VE ALTYAPI ===

3. FIBER İNTERNET PAKETLERİ

3.1 Vodafone Fiber 100 Mbps Starter
Aylık Ücret:...

✅ Vector stores read

# 🎯 SECTION 8: BGE RERANKER INITIALIZATION (OPEN SOURCE)

In [11]:
print("\n" + "="*80)
print("🎯 BGE RERANKER - OPEN SOURCE IMPLEMENTATION")
print("="*80)

if not BGE_AVAILABLE:
    print("⚠️ BGE Reranker not available. Installing...")
    !pip install -q FlagEmbedding
    from FlagEmbedding import FlagReranker
    BGE_AVAILABLE = True

print("""
📚 AVAILABLE BGE RERANKER MODELS:

1. BAAI/bge-reranker-base
   Size: ~280MB | Speed: ⚡⚡⚡ | Quality: ⭐⭐⭐

2. BAAI/bge-reranker-large
   Size: ~560MB | Speed: ⚡⚡ | Quality: ⭐⭐⭐⭐

3. BAAI/bge-reranker-v2-m3 (RECOMMENDED) 🔥
   Size: ~560MB | Speed: ⚡⚡ | Quality: ⭐⭐⭐⭐
   Languages: 100+ (Turkish ✅)

4. BAAI/bge-reranker-v2-gemma
   Size: ~2GB | Speed: ⚡ | Quality: ⭐⭐⭐⭐⭐

🎯 VODAFONE RECOMMENDATION: bge-reranker-v2-m3
   ✅ Excellent multilingual support (Turkish + English)
   ✅ Good performance/speed balance
   ✅ Production-ready
   ✅ 100% FREE & Open Source
""")

# Initialize BGE Reranker
print("\n🔄 Loading BGE Reranker v2-m3...")



🎯 BGE RERANKER - OPEN SOURCE IMPLEMENTATION

📚 AVAILABLE BGE RERANKER MODELS:

1. BAAI/bge-reranker-base
   Size: ~280MB | Speed: ⚡⚡⚡ | Quality: ⭐⭐⭐

2. BAAI/bge-reranker-large
   Size: ~560MB | Speed: ⚡⚡ | Quality: ⭐⭐⭐⭐

3. BAAI/bge-reranker-v2-m3 (RECOMMENDED) 🔥
   Size: ~560MB | Speed: ⚡⚡ | Quality: ⭐⭐⭐⭐
   Languages: 100+ (Turkish ✅)

4. BAAI/bge-reranker-v2-gemma
   Size: ~2GB | Speed: ⚡ | Quality: ⭐⭐⭐⭐⭐

🎯 VODAFONE RECOMMENDATION: bge-reranker-v2-m3
   ✅ Excellent multilingual support (Turkish + English)
   ✅ Good performance/speed balance
   ✅ Production-ready
   ✅ 100% FREE & Open Source


🔄 Loading BGE Reranker v2-m3...


In [12]:


class BGEReranker:
    """
    Production-ready BGE Reranker wrapper
    100% Open Source - Zero API costs!
    """

    def __init__(
        self,
        model_name: str = 'BAAI/bge-reranker-v2-m3',
        use_fp16: bool = True,
        device: str = None
    ):
        """
        Initialize BGE Reranker

        Args:
            model_name: BGE model name from HuggingFace
            use_fp16: Use half-precision for 2x speed
            device: 'cuda' for GPU, 'cpu' for CPU (auto-detect if None)
        """
        self.model_name = model_name
        self.use_fp16 = use_fp16

        print(f"   🔄 Loading {model_name}...")
        print(f"   ⚙️  FP16: {use_fp16}")

        try:
            # Determine device
            import torch
            if device is None:
                device = "cuda" if torch.cuda.is_available() else "cpu"

            self.device = device
            print(f"   🖥️  Device: {device}")

            # Load model
            self.reranker = FlagReranker(
                model_name,
                use_fp16=use_fp16,
                device=device
            )

            print(f"   ✅ BGE Reranker loaded successfully!")

        except Exception as e:
            print(f"   ❌ Error loading BGE Reranker: {e}")
            raise

    def rerank_documents(
        self,
        query: str,
        documents: List[Document],
        top_k: int = 5,
        normalize: bool = True
    ) -> List[Tuple[Document, float]]:
        """
        Rerank documents using BGE

        Args:
            query: Search query
            documents: Documents to rerank
            top_k: Number of top results
            normalize: Normalize scores to [0,1]

        Returns:
            List of (document, score) tuples sorted by relevance
        """
        if not documents:
            return []

        # Create query-document pairs
        pairs = [[query, doc.page_content] for doc in documents]

        # Compute scores
        scores = self.reranker.compute_score(pairs, normalize=normalize)

        # Handle single document case
        if not isinstance(scores, list):
            scores = [scores]

        # Combine documents with scores
        doc_score_pairs = list(zip(documents, scores))

        # Sort by score (descending)
        doc_score_pairs.sort(key=lambda x: x[1], reverse=True)

        return doc_score_pairs[:top_k]

    def rerank_texts(
        self,
        query: str,
        texts: List[str],
        top_k: int = 5,
        normalize: bool = True
    ) -> List[Tuple[str, float]]:
        """Rerank plain text passages"""
        if not texts:
            return []

        pairs = [[query, text] for text in texts]
        scores = self.reranker.compute_score(pairs, normalize=normalize)

        if not isinstance(scores, list):
            scores = [scores]

        text_score_pairs = list(zip(texts, scores))
        text_score_pairs.sort(key=lambda x: x[1], reverse=True)

        return text_score_pairs[:top_k]

# Initialize BGE Reranker
bge_reranker = BGEReranker(
    model_name='BAAI/bge-reranker-v2-m3',
    use_fp16=True
)


   🔄 Loading BAAI/bge-reranker-v2-m3...
   ⚙️  FP16: True
   🖥️  Device: cuda


tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

   ✅ BGE Reranker loaded successfully!


In [13]:
# Test BGE Reranker
print("\n🧪 Testing BGE Reranker...")

test_query_rerank = "Kurumsal güvenlik çözümleri ve fiyatları"
test_passages = [
    "Vodafone Genç Tarifesi 175 TL/ay, 20 GB internet içerir.",
    "Enterprise Security Suite 5,000 TL/ay, SIEM, DLP, threat protection içerir.",
    "Vodafone Fiber 1000 Mbps Ultimate 850 TL/ay simetrik gigabit sunuyor.",
    "Kurumsal güvenlik paketi VPN, firewall ve MDM içerir.",
]

# Create mock documents
test_docs = [Document(page_content=p, metadata={}) for p in test_passages]

# Rerank
reranked = bge_reranker.rerank_documents(
    query=test_query_rerank,
    documents=test_docs,
    top_k=4,
    normalize=True
)

print(f"\n   📊 Reranking Results for: '{test_query_rerank}'")
for i, (doc, score) in enumerate(reranked, 1):
    print(f"\n   {i}. Score: {score:.4f}")
    print(f"      {doc.page_content}")

print("\n✅ BGE Reranker is production-ready!")



🧪 Testing BGE Reranker...


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



   📊 Reranking Results for: 'Kurumsal güvenlik çözümleri ve fiyatları'

   1. Score: 0.6053
      Kurumsal güvenlik paketi VPN, firewall ve MDM içerir.

   2. Score: 0.0756
      Enterprise Security Suite 5,000 TL/ay, SIEM, DLP, threat protection içerir.

   3. Score: 0.0000
      Vodafone Fiber 1000 Mbps Ultimate 850 TL/ay simetrik gigabit sunuyor.

   4. Score: 0.0000
      Vodafone Genç Tarifesi 175 TL/ay, 20 GB internet içerir.

✅ BGE Reranker is production-ready!


# 🔥 SECTION 9: HYBRID SEARCH (BM25 + Semantic)

In [14]:
print("\n" + "="*80)
print("🔥 HYBRID SEARCH: BM25 + SEMANTIC SEARCH")
print("="*80)

# BM25 Retriever (Keyword-based)
print("\n1️⃣ Creating BM25 Retriever...")
bm25_retriever = BM25Retriever.from_documents(enriched_docs)
bm25_retriever.k = 10
print(f"   ✅ BM25 Retriever ready (k={bm25_retriever.k})")

# Semantic Retriever
print("\n2️⃣ Creating Semantic Retriever...")
semantic_retriever = faiss_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}
)
print(f"   ✅ Semantic Retriever ready (k=10)")

# Ensemble Retriever (Hybrid)
print("\n3️⃣ Creating Ensemble (Hybrid) Retriever...")
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, semantic_retriever],
    weights=[0.3, 0.7]  # BM25: 30%, Semantic: 70%
)
print(f"   ✅ Hybrid Retriever ready")
print(f"   ⚖️  Weights: BM25=0.3, Semantic=0.7")

# Test Hybrid Search
print("\n4️⃣ Testing Hybrid Search...")

test_queries_hybrid = [
    "Vodafone Business Pro özellikleri ve fiyatı",
    "En hızlı fiber internet paketi hangisi?",
    "IoT endüstriyel çözümler nasıl çalışır?"
]

for query in test_queries_hybrid:
    print(f"\n   {'='*76}")
    print(f"   ❓ Query: {query}")
    print(f"   {'='*76}")

    # BM25 only
    bm25_docs = bm25_retriever.invoke(query)
    print(f"\n   📊 BM25 Only (Keyword):")
    print(f"      Top: {bm25_docs[0].page_content[:100]}...")

    # Semantic only
    semantic_docs = semantic_retriever.invoke(query)
    print(f"\n   📊 Semantic Only (Embedding):")
    print(f"      Top: {semantic_docs[0].page_content[:100]}...")

    # Hybrid
    hybrid_docs = ensemble_retriever.invoke(query)
    print(f"\n   🔥 HYBRID (Best of Both):")
    print(f"      Top: {hybrid_docs[0].page_content[:100]}...")

print("\n✅ Hybrid search validated!")




🔥 HYBRID SEARCH: BM25 + SEMANTIC SEARCH

1️⃣ Creating BM25 Retriever...
   ✅ BM25 Retriever ready (k=10)

2️⃣ Creating Semantic Retriever...
   ✅ Semantic Retriever ready (k=10)

3️⃣ Creating Ensemble (Hybrid) Retriever...
   ✅ Hybrid Retriever ready
   ⚖️  Weights: BM25=0.3, Semantic=0.7

4️⃣ Testing Hybrid Search...

   ❓ Query: Vodafone Business Pro özellikleri ve fiyatı

   📊 BM25 Only (Keyword):
      Top: 1.3 Vodafone Aile Paketi
Aylık Ücret: 600 TL (4 hatta kadar)
Özellikler:
- Her hat için 30 GB İntern...

   📊 Semantic Only (Embedding):
      Top: 1.3 Vodafone Aile Paketi
Aylık Ücret: 600 TL (4 hatta kadar)
Özellikler:
- Her hat için 30 GB İntern...

   🔥 HYBRID (Best of Both):
      Top: 1.3 Vodafone Aile Paketi
Aylık Ücret: 600 TL (4 hatta kadar)
Özellikler:
- Her hat için 30 GB İntern...

   ❓ Query: En hızlı fiber internet paketi hangisi?

   📊 BM25 Only (Keyword):
      Top: 10.3 Öğrenci Avantajları
- Vodafone Genç'e ek %15 indirim
- Campus Store'larda özel kampanyalar
-

# 🎯 SECTION 10: HYBRID SEARCH + BGE RERANKING (PRODUCTION)

In [15]:
print("\n" + "="*80)
print("🎯 HYBRID SEARCH + BGE RERANKING - PRODUCTION PIPELINE")
print("="*80)

class HybridSearchWithBGE:
    """
    Production-ready Hybrid Search with BGE Reranking

    Pipeline:
    1. BM25 + Semantic Search → Retrieve top-N candidates
    2. BGE Reranking → Refine to top-K best results

    100% Open Source!
    """

    def __init__(
        self,
        vectorstore: FAISS,
        documents: List[Document],
        bge_reranker: BGEReranker,
        bm25_weight: float = 0.3,
        semantic_weight: float = 0.7
    ):
        self.vectorstore = vectorstore
        self.bge_reranker = bge_reranker

        # BM25 Retriever
        self.bm25_retriever = BM25Retriever.from_documents(documents)

        # Semantic Retriever
        self.semantic_retriever = vectorstore.as_retriever()

        # Ensemble Retriever
        self.ensemble_retriever = EnsembleRetriever(
            retrievers=[self.bm25_retriever, self.semantic_retriever],
            weights=[bm25_weight, semantic_weight]
        )

        print(f"   ✅ Hybrid Search + BGE Pipeline initialized")
        print(f"      ⚖️  BM25 Weight: {bm25_weight}")
        print(f"      ⚖️  Semantic Weight: {semantic_weight}")
        print(f"      🎯 Reranker: {bge_reranker.model_name}")

    def search(
        self,
        query: str,
        top_k: int = 5,
        initial_k: int = 20,
        use_reranking: bool = True
    ) -> List[Tuple[Document, float]]:
        """
        Hybrid search with optional BGE reranking

        Args:
            query: Search query
            top_k: Final number of results
            initial_k: Initial retrieval size
            use_reranking: Whether to apply BGE reranking

        Returns:
            List of (document, score) tuples
        """
        # Step 1: Hybrid retrieval
        self.bm25_retriever.k = initial_k
        self.semantic_retriever.search_kwargs = {"k": initial_k}

        hybrid_docs = self.ensemble_retriever.invoke(query)

        # Step 2: BGE Reranking (optional)
        if use_reranking and len(hybrid_docs) > 0:
            reranked = self.bge_reranker.rerank_documents(
                query=query,
                documents=hybrid_docs,
                top_k=top_k,
                normalize=True
            )
            return reranked
        else:
            # Return top-k without reranking
            return [(doc, 0.0) for doc in hybrid_docs[:top_k]]

# Initialize Production Hybrid Search
print("\n🚀 Initializing Production Hybrid Search + BGE...")

hybrid_bge_system = HybridSearchWithBGE(
    vectorstore=faiss_vectorstore,
    documents=enriched_docs,
    bge_reranker=bge_reranker,
    bm25_weight=0.3,
    semantic_weight=0.7
)


🎯 HYBRID SEARCH + BGE RERANKING - PRODUCTION PIPELINE

🚀 Initializing Production Hybrid Search + BGE...
   ✅ Hybrid Search + BGE Pipeline initialized
      ⚖️  BM25 Weight: 0.3
      ⚖️  Semantic Weight: 0.7
      🎯 Reranker: BAAI/bge-reranker-v2-m3


In [16]:
# Test Production Pipeline
print("\n🧪 Testing Production Pipeline...")

test_queries_production = [
    "En uygun fiyatlı mobil paket hangisi?",
    "Kurumsal müşteriler için güvenlik çözümleri",
    "5G kapsama alanı nedir?"
]

for query in test_queries_production:
    print(f"\n{'='*80}")
    print(f"❓ Query: {query}")
    print(f"{'='*80}")

    # With reranking
    results = hybrid_bge_system.search(
        query=query,
        top_k=3,
        initial_k=15,
        use_reranking=True
    )

    print(f"\n🎯 Top 3 Results (Hybrid + BGE Reranked):\n")
    for i, (doc, score) in enumerate(results, 1):
        print(f"{i}. BGE Score: {score:.4f}")
        print(f"   Categories: {doc.metadata.get('categories', [])}")
        print(f"   Section: {doc.metadata.get('section', 'N/A')}")
        print(f"   Content: {doc.page_content[:150]}...")
        print()

print("✅ Production pipeline validated!")



🧪 Testing Production Pipeline...

❓ Query: En uygun fiyatlı mobil paket hangisi?

🎯 Top 3 Results (Hybrid + BGE Reranked):

1. BGE Score: 0.2018
   Categories: ['mobile', 'business', 'security', 'cloud', 'campaign', 'support']
   Section: İKİNCİ BÖLÜM: İNTERNET VE ALTYAPI
   Content: 1.3 Vodafone Aile Paketi
Aylık Ücret: 600 TL (4 hatta kadar)
Özellikler:
- Her hat için 30 GB İnternet (Toplam 120 GB paylaşımlı)
- Hatlar arası sınır...

2. BGE Score: 0.1891
   Categories: ['mobile', 'tv', 'cloud', 'campaign', 'support']
   Section: 
   Content: VODAFONE TÜRKİYE KAPSAMLI ÜRÜN VE HİZMET KATALOĞU 2024-2025

=== BİRİNCİ BÖLÜM: MOBİL PAKE...

3. BGE Score: 0.0614
   Categories: ['mobile', 'tv', 'business', 'security', 'cloud']
   Section: Unknown
   Content: 5.3 Ultimate Entertainment - 250 TL/ay
- Tüm Kanallar (150+)
- Tüm Spor Paketleri
- HBO Max, Disney+ Dahil
- Exclusive Vodafone Content
- 8 Eşzamanlı ...


❓ Query: Kurumsal müşteriler için güvenlik çözümleri

🎯 Top 3 Results (Hybrid + 

# 🧠 SECTION 11: MULTI-QUERY RETRIEVAL

In [17]:
print("\n" + "="*80)
print("🧠 MULTI-QUERY RETRIEVAL")
print("="*80)

print("""
Multi-Query Retrieval expands a single query into multiple perspectives
to improve retrieval coverage and quality.

Example:
User: "En ucuz internet paketi"
LLM generates:
- "Vodafone'un en ekonomik fiber internet paketi hangisi?"
- "Düşük fiyatlı internet seçenekleri neler?"
- "Bütçe dostu internet paketleri"
""")

# Initialize Multi-Query Retriever
print("\n🔄 Creating Multi-Query Retriever...")

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=faiss_vectorstore.as_retriever(search_kwargs={"k": 5}),
    llm=llm
)

print("   ✅ Multi-Query Retriever ready")

# Test
print("\n🧪 Testing Multi-Query Retrieval...")

test_query_multi = "Öğrenciler için uygun paketler"

print(f"   📝 Original Query: '{test_query_multi}'")
print("\n   🔄 LLM generating query variations...")

multi_docs = multi_query_retriever.invoke(test_query_multi)

print(f"\n   ✅ Retrieved {len(multi_docs)} unique documents")
print("\n   📊 Top 3 Results:")

for i, doc in enumerate(multi_docs[:3], 1):
    print(f"\n   {i}. Categories: {doc.metadata.get('categories', [])}")
    print(f"      Content: {doc.page_content[:120]}...")

print("\n✅ Multi-Query Retrieval validated!")



🧠 MULTI-QUERY RETRIEVAL

Multi-Query Retrieval expands a single query into multiple perspectives
to improve retrieval coverage and quality.

Example:
User: "En ucuz internet paketi"
LLM generates:
- "Vodafone'un en ekonomik fiber internet paketi hangisi?"
- "Düşük fiyatlı internet seçenekleri neler?"
- "Bütçe dostu internet paketleri"


🔄 Creating Multi-Query Retriever...
   ✅ Multi-Query Retriever ready

🧪 Testing Multi-Query Retrieval...
   📝 Original Query: 'Öğrenciler için uygun paketler'

   🔄 LLM generating query variations...

   ✅ Retrieved 5 unique documents

   📊 Top 3 Results:

   1. Categories: ['mobile', 'tv', 'cloud', 'campaign', 'support']
      Content: VODAFONE TÜRKİYE KAPSAMLI ÜRÜN VE HİZMET KATALOĞU 2024-2025
...

   2. Categories: ['mobile', 'business', 'security', 'cloud', 'campaign', 'support']
      Content: 1.3 Vodafone Aile Paketi
Aylık Ücret: 600 TL (4 hatta kadar)
Özellikler:
- Her hat için 30 GB İnternet (Toplam 120 GB pa...

   3. Categories: ['mobile', 't

# 🌳 SECTION 12: PARENT DOCUMENT RETRIEVAL

In [18]:
print("\n" + "="*80)
print("🌳 PARENT DOCUMENT RETRIEVAL")
print("="*80)

print("""
Parent Document Retrieval:
- Creates small chunks for precise embedding/matching
- Returns larger parent chunks for better context
- Best of both worlds: precision + context
""")

# Store for parent documents
print("\n🔄 Setting up Parent Document Retriever...")

parent_store = InMemoryStore()

# Child splitter (small chunks for embedding)
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50
)

# Parent splitter (larger chunks to return)
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200
)

# Create new vectorstore for parent retrieval
parent_vectorstore = FAISS.from_documents(
    documents=[Document(page_content="")],  # Placeholder
    embedding=embeddings
)

# Create Parent Document Retriever
parent_retriever = ParentDocumentRetriever(
    vectorstore=parent_vectorstore,
    docstore=parent_store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

# Add documents
print("   🔄 Processing documents...")
parent_retriever.add_documents(documents)

print(f"   ✅ Parent Document Retriever ready")
print(f"      📏 Child chunk size: ~400 chars (for embedding)")
print(f"      📏 Parent chunk size: ~2000 chars (for context)")



🌳 PARENT DOCUMENT RETRIEVAL

Parent Document Retrieval:
- Creates small chunks for precise embedding/matching
- Returns larger parent chunks for better context
- Best of both worlds: precision + context


🔄 Setting up Parent Document Retriever...
   🔄 Processing documents...
   ✅ Parent Document Retriever ready
      📏 Child chunk size: ~400 chars (for embedding)
      📏 Parent chunk size: ~2000 chars (for context)


In [19]:
# Test
print("\n🧪 Testing Parent Document Retrieval...")

test_query_parent = "Vodafone Business Pro detayları"
parent_docs = parent_retriever.invoke(test_query_parent)

print(f"\n   📊 Retrieved {len(parent_docs)} parent documents")
print(f"\n   📄 First parent document (full context):")
print(f"   {parent_docs[0].page_content[:600]}...")

print("\n✅ Parent Document Retrieval validated!")


🧪 Testing Parent Document Retrieval...

   📊 Retrieved 2 parent documents

   📄 First parent document (full context):
   VODAFONE TÜRKİYE KAPSAMLI ÜRÜN VE HİZMET KATALOĞU 2024-2025

=== BİRİNCİ BÖLÜM: MOBİL PAKETLER ===

1. CONSUMER PAKETLERİ

1.1 Vodafone Genç Tarifesi Plus
Hedef Kitle: 18-28 yaş arası genç kullanıcılar
Aylık Ücret: 175 TL
Özellikler:
- 20 GB Genel İnternet Paketi
- Sosyal Medya Uygulamalarına Sınırsız İnternet (Instagram, TikTok, Twitter)
- Müzik Streaming Sınırsız (Spotify, Apple Music)
- Video Streaming 30 GB (YouTube, Netflix)
- Sınırsız Konuşma ve SMS
- 5G Desteği
- Uluslararası 100 Dakika
- Gaming Mode: Düşük ping garantisi
- Vod...

✅ Parent Document Retrieval validated!


# 🤖 SECTION 13: AGENT-BASED RAG WITH TOOLS

In [20]:
print("\n" + "="*80)
print("🤖 AGENT-BASED RAG WITH TOOLS")
print("="*80)

print("""
Agent-based RAG allows:
- Dynamic tool selection
- Multi-step reasoning
- Complex query handling
- Action-based responses
""")


🤖 AGENT-BASED RAG WITH TOOLS

Agent-based RAG allows:
- Dynamic tool selection
- Multi-step reasoning
- Complex query handling
- Action-based responses



In [21]:
print("\n" + "="*80)
print("🤖 AGENT-BASED RAG WITH TOOLS")
print("="*80)

print("""
Agent-based RAG enables:
- Dynamic tool selection
- Multi-step reasoning
- Complex query handling
- Action-based responses
""")

# Import tool decorator
from langchain_core.tools import tool

# Define RAG Search Tool
@tool
def vodafone_search(query: str) -> str:
    """
    Search Vodafone catalog for product and service information.

    Use this tool to find information about:
    - Mobile packages and pricing
    - Internet services and speeds
    - IoT solutions
    - Campaigns and promotions
    - Technical specifications

    Args:
        query: Search query describing what information is needed

    Returns:
        Relevant information from Vodafone catalog
    """
    try:
        # Use hybrid search with BGE reranking
        results = hybrid_bge_system.search(
            query=query,
            top_k=3,
            initial_k=10,
            use_reranking=True
        )

        if not results:
            return "İlgili bilgi bulunamadı. Lütfen sorgunuzu farklı kelimelerle ifade edin."

        # Format results
        context_parts = []
        for i, (doc, score) in enumerate(results, 1):
            context_parts.append(
                f"[Kaynak {i} - Skor: {score:.2f}]\n{doc.page_content[:300]}"
            )

        return "\n\n".join(context_parts)

    except Exception as e:
        return f"Arama sırasında hata oluştu: {str(e)}"

# Define Price Calculator Tool
@tool
def calculate_package_price(package_names: str) -> str:
    """
    Calculate total monthly price for multiple Vodafone packages.

    Use this tool to:
    - Calculate combined package costs
    - Compare pricing options
    - Get budget estimates

    Args:
        package_names: Comma-separated list of package names
                      Example: "Vodafone Genç, Fiber 100"

    Returns:
        Detailed price breakdown and total cost
    """
    try:
        packages = [p.strip() for p in package_names.split(',')]

        total = 0
        details = []

        for package in packages:
            # Search for package price
            docs = faiss_vectorstore.similarity_search(
                f"{package} ücret fiyat TL",
                k=2
            )

            if docs:
                content = docs[0].page_content

                # Extract price using regex
                import re
                prices = re.findall(r'(\d+)\s*TL', content)

                if prices:
                    price = int(prices[0])
                    total += price
                    details.append(f"✓ {package}: {price} TL/ay")
                else:
                    details.append(f"? {package}: Fiyat bulunamadı")
            else:
                details.append(f"✗ {package}: Ürün bulunamadı")

        result = "💰 Fiyat Hesaplama:\n\n"
        result += "\n".join(details)
        result += f"\n\n📊 Toplam Aylık Maliyet: {total} TL"

        return result

    except Exception as e:
        return f"Fiyat hesaplama hatası: {str(e)}"

# Define Package Comparison Tool
@tool
def compare_packages(package1: str, package2: str) -> str:
    """
    Compare two Vodafone packages side-by-side.

    Use this tool to:
    - Compare features between packages
    - Help customers choose the right package
    - Highlight differences and similarities

    Args:
        package1: First package name
        package2: Second package name

    Returns:
        Detailed comparison of the two packages
    """
    try:
        # Search for both packages
        docs1 = faiss_vectorstore.similarity_search(f"{package1} özellik", k=1)
        docs2 = faiss_vectorstore.similarity_search(f"{package2} özellik", k=1)

        if not docs1 or not docs2:
            return "Paketlerden biri veya her ikisi bulunamadı."

        comparison = f"📊 PAKET KARŞILAŞTIRMA\n\n"
        comparison += f"1️⃣ {package1}:\n{docs1[0].page_content[:250]}...\n\n"
        comparison += f"2️⃣ {package2}:\n{docs2[0].page_content[:250]}...\n\n"
        comparison += "💡 Daha detaylı bilgi için ilgili paketi ayrıca sorgulayabilirsiniz."

        return comparison

    except Exception as e:
        return f"Karşılaştırma hatası: {str(e)}"

# Collect all tools
agent_tools = [
    vodafone_search,
    calculate_package_price,
    compare_packages
]

print(f"\n   🛠️  Defined {len(agent_tools)} tools:")
for tool_item in agent_tools:
    print(f"      - {tool_item.name}: {tool_item.description.split('.')[0]}")


🤖 AGENT-BASED RAG WITH TOOLS

Agent-based RAG enables:
- Dynamic tool selection
- Multi-step reasoning
- Complex query handling
- Action-based responses


   🛠️  Defined 3 tools:
      - vodafone_search: Search Vodafone catalog for product and service information
      - calculate_package_price: Calculate total monthly price for multiple Vodafone packages
      - compare_packages: Compare two Vodafone packages side-by-side


In [23]:
# Create Agent using LangChain v1.x compatible approach
print("\n   🔄 Creating agent with bind_tools approach (LangChain v1.x)...")

# Bind tools to LLM
llm_with_tools = llm.bind_tools(agent_tools)

# Agent prompt
agent_system_prompt = """Sen Vodafone Türkiye'nin profesyonel AI asistanısın.

Görevin müşterilere ürün ve hizmetler hakkında doğru bilgi vermek.

Kullanabileceğin araçlar:
- vodafone_search: Ürün/hizmet bilgilerini aramak için kullan
- calculate_package_price: Toplam maliyet hesaplamak için kullan
- compare_packages: İki paketi karşılaştırmak için kullan

Önemli kurallar:
1. Önce ilgili aracı kullan
2. Sonra bulduğun bilgiyi profesyonel Türkçe ile açıkla
3. Fiyatları net ve doğru belirt
4. Kısa ve öz cevaplar ver"""

# Enhanced agent function with detailed logging
def simple_agent(user_input: str, max_iterations: int = 3, verbose: bool = True) -> str:
    """
    Simple agent that uses tools iteratively with detailed logging

    Args:
        user_input: User's question
        max_iterations: Maximum tool call iterations
        verbose: Show detailed tool execution logs

    Returns:
        Final answer
    """
    messages = [
        {"role": "system", "content": agent_system_prompt},
        {"role": "user", "content": user_input}
    ]

    if verbose:
        print(f"   🤔 Agent başlıyor...")

    for iteration in range(max_iterations):
        if verbose:
            print(f"\n   🔄 Iteration {iteration + 1}/{max_iterations}")

        # Get LLM response
        response = llm_with_tools.invoke(messages)

        # Check if there are tool calls
        if not response.tool_calls:
            # No more tool calls, return final answer
            if verbose:
                print(f"   ✅ Final answer generated (no more tool calls)")
            return response.content

        # Log tool calls
        if verbose:
            print(f"\n   🛠️  Tool Calls Detected: {len(response.tool_calls)}")

        # Execute tool calls
        messages.append(response)

        for idx, tool_call in enumerate(response.tool_calls, 1):
            tool_name = tool_call['name']
            tool_args = tool_call['args']

            if verbose:
                print(f"\n   📌 Tool {idx}: {tool_name}")
                print(f"      Arguments: {json.dumps(tool_args, ensure_ascii=False, indent=6)}")

            # Find and execute the tool
            tool_result = None
            for tool in agent_tools:
                if tool.name == tool_name:
                    try:
                        if verbose:
                            print(f"      ⏳ Executing {tool_name}...")

                        tool_result = tool.invoke(tool_args)

                        if verbose:
                            print(f"      ✅ Tool executed successfully")
                            print(f"      📊 Result preview: {tool_result[:150]}...")
                    except Exception as e:
                        tool_result = f"Tool error: {str(e)}"
                        if verbose:
                            print(f"      ❌ Tool error: {str(e)}")
                    break

            if tool_result is None:
                tool_result = f"Tool {tool_name} not found"
                if verbose:
                    print(f"      ⚠️  Tool not found: {tool_name}")

            # Add tool result to messages
            messages.append({
                "role": "tool",
                "content": tool_result,
                "tool_call_id": tool_call.get('id', 'unknown')
            })

    # Max iterations reached, return last response
    if verbose:
        print(f"\n   ⚠️  Max iterations ({max_iterations}) reached")

    final_response = llm_with_tools.invoke(messages)
    return final_response.content

print("   ✅ Agent created successfully!")
print("   🎯 Using bind_tools + manual tool execution (v1.x compatible)")
print("   📝 Verbose logging enabled")

# Test Agent
print("\n🧪 Testing Agent-based RAG with Detailed Logging...")

test_queries_agent = [
    "Vodafone Business Pro paketinin özellikleri nedir?",
    "Vodafone Genç Tarifesi ve Fiber 500 Mbps toplam ne kadar?",
    "IoT endüstriyel çözümler nedir?"
]

for query_idx, query in enumerate(test_queries_agent, 1):
    print(f"\n{'='*80}")
    print(f"TEST {query_idx}/{len(test_queries_agent)}")
    print(f"{'='*80}")
    print(f"👤 User: {query}")
    print(f"{'='*80}")

    try:
        # Invoke agent with verbose logging
        answer = simple_agent(query, max_iterations=3, verbose=True)

        print(f"\n   {'='*76}")
        print(f"   🤖 FINAL ANSWER:")
        print(f"   {'='*76}")
        print(f"   {answer}")
        print(f"   {'='*76}\n")

    except Exception as e:
        print(f"\n   ❌ Agent error: {str(e)}\n")
        import traceback
        traceback.print_exc()

print("✅ Agent-based RAG validated!")

# Alternative: Direct tool usage with logging
print("\n" + "="*80)
print("📌 ALTERNATIVE: Direct Tool Usage (Simplified with Logging)")
print("="*80)

def query_with_tools(user_query: str, verbose: bool = True) -> str:
    """
    Simplified approach: Use LLM to decide which tool to call
    then execute directly with logging
    """
    if verbose:
        print(f"\n   🤔 Analyzing query...")

    # Ask LLM which tool to use
    decision_prompt = f"""Kullanıcı sorusu: {user_query}

Mevcut araçlar:
1. vodafone_search - Ürün/hizmet araması için
2. calculate_package_price - Fiyat hesaplama için
3. compare_packages - Paket karşılaştırma için

Hangi aracı kullanmalıyım ve hangi parametrelerle? JSON formatında cevap ver:
{{"tool": "tool_name", "params": {{"param1": "value1"}}}}"""

    decision = llm.invoke([HumanMessage(content=decision_prompt)])

    if verbose:
        print(f"   📋 Tool decision received")

    try:
        import json
        # Try to parse JSON response
        decision_content = decision.content.strip('```json\n').strip('```').strip()
        decision_json = json.loads(decision_content)
        tool_name = decision_json.get('tool')
        params = decision_json.get('params', {})

        if verbose:
            print(f"\n   🛠️  Selected Tool: {tool_name}")
            print(f"   📝 Parameters: {json.dumps(params, ensure_ascii=False, indent=6)}")

        # Execute tool
        for tool in agent_tools:
            if tool.name == tool_name:
                if verbose:
                    print(f"   ⏳ Executing {tool_name}...")

                result = tool.invoke(params)

                if verbose:
                    print(f"   ✅ Tool executed successfully")
                    print(f"   📊 Result preview: {result[:150]}...")

                # Generate final answer
                final_prompt = f"""Kullanıcı sorusu: {user_query}

Araç sonucu:
{result}

Profesyonel ve kısa bir cevap ver (Türkçe):"""

                if verbose:
                    print(f"   🤖 Generating final answer...")

                final_answer = llm.invoke([HumanMessage(content=final_prompt)])

                if verbose:
                    print(f"   ✅ Answer generated")

                return final_answer.content

        if verbose:
            print(f"   ⚠️  Tool '{tool_name}' not found, falling back to search")

        return "Uygun araç bulunamadı."

    except Exception as e:
        if verbose:
            print(f"   ⚠️  Error parsing decision, using fallback: {str(e)}")

        # Fallback: Direct search
        if verbose:
            print(f"   🛠️  Fallback: Using vodafone_search")
            print(f"   ⏳ Executing...")

        result = vodafone_search.invoke({"query": user_query})

        if verbose:
            print(f"   ✅ Search completed")

        final_prompt = f"""Soru: {user_query}

Bilgi:
{result}

Kısa cevap ver:"""
        final_answer = llm.invoke([HumanMessage(content=final_prompt)])
        return final_answer.content

# Test simple approach
print("\n🧪 Testing Direct Tool Usage with Logging...")

simple_test_query = "En ucuz mobil paket hangisi?"
print(f"\n{'='*80}")
print(f"❓ Query: {simple_test_query}")
print(f"{'='*80}")

simple_result = query_with_tools(simple_test_query, verbose=True)

print(f"\n{'='*80}")
print(f"💡 FINAL RESPONSE:")
print(f"{'='*80}")
print(f"{simple_result}")
print(f"{'='*80}\n")

print("✅ Direct tool usage validated!")

# Summary
print("\n" + "="*80)
print("📊 AGENT SYSTEM SUMMARY")
print("="*80)

summary = f"""
✅ Agent Configuration:
   • Tools Available: {len(agent_tools)}
   • Max Iterations: 3
   • Verbose Logging: ✅ Enabled

🛠️  Available Tools:
"""

for i, tool_item in enumerate(agent_tools, 1):
    summary += f"   {i}. {tool_item.name}\n"
    summary += f"      └─ {tool_item.description.split('.')[0]}\n"

summary += """
📝 Logging Features:
   ✅ Tool selection tracking
   ✅ Parameter display
   ✅ Execution status
   ✅ Result previews
   ✅ Error reporting

🎯 Agent Flow:
   1. User query received
   2. LLM decides tool(s) to use
   3. Tool execution with logging
   4. Results processed
   5. Final answer generated

💡 Production Ready: ✅
"""

print(summary)


   🔄 Creating agent with bind_tools approach (LangChain v1.x)...
   ✅ Agent created successfully!
   🎯 Using bind_tools + manual tool execution (v1.x compatible)
   📝 Verbose logging enabled

🧪 Testing Agent-based RAG with Detailed Logging...

TEST 1/3
👤 User: Vodafone Business Pro paketinin özellikleri nedir?
   🤔 Agent başlıyor...

   🔄 Iteration 1/3

   🛠️  Tool Calls Detected: 1

   📌 Tool 1: vodafone_search
      Arguments: {
      "query": "Vodafone Business Pro paketi özellikleri"
}
      ⏳ Executing vodafone_search...
      ✅ Tool executed successfully
      📊 Result preview: [Kaynak 1 - Skor: 0.99]
1.3 Vodafone Aile Paketi
Aylık Ücret: 600 TL (4 hatta kadar)
Özellikler:
- Her hat için 30 GB İnternet (Toplam 120 GB paylaşım...

   🔄 Iteration 2/3
   ✅ Final answer generated (no more tool calls)

   🤖 FINAL ANSWER:
   Vodafone Business Pro paketi hakkında doğrudan bilgi bulunmamaktadır. Başka bir paket veya hizmet hakkında bilgi almak ister misiniz?


TEST 2/3
👤 User: Vodafone 

# FINAL SUMMARY & BEST PRACTICES

---

### ✅ PRODUCTION RAG BEST PRACTICES (2025 Edition)

---

## 1) 🔐 SECURITY & COMPLIANCE (CRITICAL!)

### ⚠️ LangChain Version
- **MUST** use **v1.2.7+**
- Check:
```bash
pip show langchain
```
- Update:
```bash
pip install --upgrade "langchain>=1.2.7"
```

### 🔐 API Key Management
- **NEVER** hardcode API keys in code
- Use **Azure Key Vault** or **environment variables**
- Rotate keys regularly (every **90 days**)
- Use separate keys for **dev / staging / production**

### 🛡️ Input Validation
- Sanitize all user inputs
- Implement rate limiting (**10–50 req/min per user**)
- Set max input length (e.g., **2000 chars**)
- Block injection attempts

### 📝 Audit & Compliance
- Log all queries and responses
- Track usage per user/department
- GDPR/KVKK compliance for data handling
- Regular security audits

---

## 2) 🏗️ RAG ARCHITECTURE (OPTIMAL SETUP)

### 📊 Retrieval Pipeline
```text
┌─────────────────────────────────────────────────┐
│ 1. Hybrid Search (BM25 + Semantic)             │
│    • BM25 Weight: 0.3 (keyword matching)       │
│    • Semantic Weight: 0.7 (meaning)            │
│    • Initial Retrieval: k=15-20                │
├─────────────────────────────────────────────────┤
│ 2. BGE Reranking (Open Source)                 │
│    • Model: BAAI/bge-reranker-v2-m3            │
│    • FP16: Enabled (2x faster)                 │
│    • Final Results: top_k=3-5                  │
├─────────────────────────────────────────────────┤
│ 3. LLM Generation                              │
│    • Temperature: 0 (consistent answers)       │
│    • Max Tokens: 1000-2000                     │
│    • Stream: True (better UX)                  │
└─────────────────────────────────────────────────┘
```

### 🎯 Why This Works
- Hybrid search catches both keywords and semantics
- BGE reranking refines to best matches
- Low temperature ensures consistency
- Total latency: **4–6 seconds** (acceptable)

---

## 3) 🔓 OPEN SOURCE RERANKING (COST OPTIMIZATION)

### 💰 Commercial vs Open Source

**Commercial Rerankers (Cohere, etc.)**
- Cost: **$1–5 per 1,000 queries**
- 100k queries/month: **$2,000–5,000/month**
- 1M queries/month: **$20,000–50,000/month**
- ✗ Vendor lock-in  
- ✗ API rate limits  
- ✗ Privacy concerns  

**BGE Reranker v2-m3 (Open Source)**
- Cost: **$0** (one-time model download ~560MB)
- Unlimited queries
- No API calls, runs locally/on-premise
- ✓ Full control  
- ✓ GDPR/KVKK compliant  
- ✓ No rate limits  
- ✓ Customizable  

### 📊 Annual Savings Example (Vodafone Scale)
- 500k queries/month → Save **~$30k–60k/year**
- 2M queries/month → Save **~$120k–240k/year**

### 🎯 BGE Model Selection
- Turkish + English → **bge-reranker-v2-m3** ⭐ RECOMMENDED
- Max Performance → bge-reranker-v2-gemma
- Fastest/Smallest → bge-reranker-base

---

## 4) 📏 TEXT SPLITTING & CHUNKING

### ✅ Optimal Configuration
- Chunk Size: **800–1200 characters**
  - Too small (< 500): Loss of context
  - Too large (> 2000): Noisy retrieval
- Overlap: **150–250 characters** (15–20%)
  - Ensures continuity
  - Prevents boundary loss
- Separators (in order):
  1. `"\n\n"` (paragraphs)
  2. `"\n"` (lines)
  3. `". "` (sentences)
  4. `" "` (words)

### 🎯 Metadata Enrichment
- Always add: categories, dates, source
- Enable filtering by metadata
- Helps with precision retrieval

```python
metadata = {
    "category": ["mobile", "pricing"],
    "date": "2025-01-15",
    "source": "vodafone_catalog",
    "has_price": True
}
```

---

## 5) 🗄️ VECTOR STORE SELECTION

### Development/POC
- ✅ FAISS (in-memory)
  - Pros: Fastest, simple, no infrastructure
  - Cons: Not persistent, single-machine
  - Use: Local dev, POC, testing

### Production (Small-Medium)
- ✅ Chroma (persistent)
  - Pros: Persistent, easy setup, good for < 1M docs
  - Cons: Single-node, limited scale
  - Use: Small–medium deployments

### Production (Large Scale)
- ✅ Qdrant / Milvus / Pinecone / Weaviate
  - Pros: Distributed, scalable, managed
  - Cons: More complex, hosting costs
  - Use: Enterprise, > 1M documents

### 🎯 Vodafone Recommendation
- Start: FAISS (development)
- MVP: Chroma (persistent, easy)
- Scale: Qdrant / Milvus (open-source, scalable)

---

## 6) 🤖 LLM & PROMPT OPTIMIZATION

### Azure OpenAI Model Selection
- GPT-4.1: Best quality, slower, $$$
- GPT-4.1 Mini: Good quality, faster, $

### ✅ Prompt Engineering

**Bad Prompt ❌**
> Answer this question using the context.

**Good Prompt ✅**
```text
Sen Vodafone Türkiye'nin AI asistanısın.

Context:
{context}

Soru: {question}

Kurallar:
1. Sadece verilen context'i kullan
2. Emin değilsen "bilmiyorum" de
3. Fiyatları tam ve doğru belirt
4. Kısa ve profesyonel cevap ver
5. Türkçe yaz

Cevap:
```

### 🎯 Temperature Settings
- 0.0: Tutarlı, deterministik (Customer Service ✓)
- 0.3–0.7: Dengeli (General Use)
- 0.8+: Yaratıcı (Content Generation)

---

## 7) 💾 CACHING & PERFORMANCE

### 🎯 Semantic Caching
- Cache query embeddings
- If similarity > 0.95 → return cached answer
- Reduces LLM calls by **30–50%**

```python
cache = {}
query_embedding = embed(query)

for cached_q, cached_a in cache.items():
    if cosine_similarity(query_embedding, cached_q) > 0.95:
        return cached_a

cache[query_embedding] = answer
```

### 💾 Result Caching
- Cache exact query → answer pairs
- TTL: **1–24 hours**
- Hit rate target: **20–40%**

### ⚡ Performance Tips
- Batch embed documents (not one-by-one)
- Use async for concurrent queries
- Index documents during off-peak hours
- Monitor p50, p95, p99 latencies

---

## 8) 🤖 AGENT-BASED RAG (LangChain v1.x)

### ✅ Use create_agent (new standard)
```python
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="Your instructions here"
)
```

### 🛠️ Tool Design Best Practices

**1) Clear Descriptions**
```python
from langchain_core.tools import tool

@tool
def search_products(query: str) -> str:
    """
    Search product catalog.

    Use when: User asks about products, prices, features
    Input: Search query (e.g., "mobile packages")
    Output: Product information
    """
    ...
```

**2) Error Handling**
```python
try:
    result = search(query)
    return result
except Exception as e:
    return f"Search failed: {str(e)}"
```

**3) Logging**
```python
print(f"🛠️ Tool: {tool_name}")
print(f"📝 Args: {args}")
print(f"✅ Result: {result[:100]}...")
```

### ⚠️ Common Pitfalls
- Too many tools (> 10) → LLM confusion
- Vague descriptions → Wrong tool selection
- Missing error handling → Agent crashes

---

## 9) 📊 MONITORING & OBSERVABILITY

### Track These Metrics

**📈 Quality**
- Retrieval Precision
- Answer Accuracy
- User Satisfaction (thumbs up/down)

**⏱️ Performance**
- Latency (p50, p95, p99)
- Throughput (qps)
- Cache hit rate

**💰 Cost**
- Embedding API calls & costs
- LLM API calls & costs
- Infrastructure costs

### 🛠️ Tools
- LangSmith/Langfuse: tracing & debugging
- Prometheus: metrics
- Grafana: dashboards
- Azure Monitor: cloud monitoring

### 🎯 Alerts
- Latency > 10s (p95)
- Error rate > 5%
- Cost > budget threshold

---

## 10) 🚀 DEPLOYMENT & SCALING

### Container Setup (Docker)
```dockerfile
FROM python:3.11-slim

COPY requirements.txt .
RUN pip install -r requirements.txt

RUN python -c "from FlagEmbedding import FlagReranker; FlagReranker('BAAI/bge-reranker-v2-m3')"

COPY . /app
WORKDIR /app

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
```

### Kubernetes
- Replicas: **3–5**
- Resources:
  - CPU: 2–4 cores per pod
  - Memory: 8–16 GB per pod
  - GPU: Optional
- Auto-scaling:
  - Min: 2, Max: 10, Target CPU: 70%

### Load Balancing
- Azure Load Balancer / Application Gateway
- Health checks: `/health`
- Sticky sessions: Optional

---

## 11) 🧪 TESTING & QUALITY ASSURANCE

### Test Categories
- ✅ Unit Tests (coverage > 80%)
- ✅ Integration Tests (50–100 Q&A pairs)
- ✅ Regression Tests (alert if accuracy drops > 10%)
- ✅ Load Tests (100–1000 concurrent users)

### 🎯 Benchmarks
- Retrieval Precision: > 70%
- Answer Accuracy: > 85%
- Latency (p95): < 5s
- Availability: > 99.5%

---

# ===================================================
# 🎯 FINAL MODULE SUMMARY
# ===================================================

## 🎯 MODÜL 3 — FINAL SUMMARY

```text
╔════════════════════════════════════════════════════════════════════════════╗
║                    ADVANCED RAG MODULE - COMPLETE                           ║
║                   Vodafone Big Data & AI Team - 2026                       ║
╚════════════════════════════════════════════════════════════════════════════╝
```

### ✅ COMPLETED TOPICS

**CORE RAG COMPONENTS**
1. ✅ LangChain v1.2.7+ (Security Patched)  
2. ✅ Azure OpenAI Integration (Latest API)  
3. ✅ Advanced Text Splitting & Metadata  
4. ✅ FAISS & Chroma Vector Stores  
5. ✅ Similarity Search & Filtering  

**ADVANCED RAG TECHNIQUES**

6. ✅ Hybrid Search (BM25 + Semantic) ⭐  
7. ✅ BGE Reranking (Open Source) ⭐⭐  
8. ✅ Multi-Query Retrieval ⭐  
9. ✅ Parent Document Retrieval ⭐   
10. ✅ Agent-based RAG with Tools 🔥  

**PRODUCTION FEATURES**

11. ✅ Production RAG System Class  
12. ✅ Caching & Performance Optimization  
13. ✅ Security Best Practices (CVE patched)  
14. ✅ Detailed Logging & Monitoring  

---

## 📊 SYSTEM ARCHITECTURE

```text
User Query
   ↓
1) Hybrid Search (BM25 30% + Semantic 70%) → top 15
   ↓
2) BGE Reranking (bge-reranker-v2-m3) → top 3–5
   ↓
3) LLM Generation (Azure OpenAI) → Final Answer
```

---

## 🔓 OPEN SOURCE BENEFITS (BGE)

```text
Commercial Rerankers:
- Cohere: $2–5 per 1k queries
- 100k/mo: $4,000–10,000
- 1M/mo:  $40,000–100,000

BGE v2-m3 (Open Source):
- $0/mo (unlimited)
- One-time: ~560MB model download
- Annual Savings: $50,000–120,000+
```

---

## 🎯 SYSTEM METRICS (Placeholders)

- Documents Processed: `len(enriched_docs)` chunks  
- Vector Store: `faiss_vectorstore.index.ntotal` vectors indexed  
- Embedding Dimension: `len(test_embeddings[0])`  
- Reranker: BGE v2-m3 (Multilingual)  
- Agent Tools: 3 (search, calculate, compare)  

---

## 🔧 TECHNOLOGY STACK

- LangChain: v1.2.7+  
- LangChain Core: v0.3.81+  
- Python: 3.10+  
- LLM: Azure OpenAI GPT-4  
- Embeddings: text-embedding-3-large  
- Reranker: BAAI/bge-reranker-v2-m3  
- Vector Stores: FAISS → Chroma → Qdrant  

---


## 💡 KEY TAKEAWAYS
1. Security first (patched versions, key mgmt, access control)
2. Cost optimization (BGE + caching + token discipline)
3. Quality (hybrid search + reranking + prompt craft)
4. Monitor everything (latency, quality, cost)
5. Start simple, scale smart (FAISS → Chroma → Qdrant)

---

# ==================================================
# 🎉 CLOSING REMARKS
# ==================================================

## 🎉 MODÜL 3 TAMAMLANDI — ADVANCED RAG EDITION!

```text
╔════════════════════════════════════════════════════════════════════════════╗
║                  🎓 VODAFONE BIG DATA & AI TEAM                            ║
║              Advanced RAG Module Successfully Completed!                   ║
╚════════════════════════════════════════════════════════════════════════════╝
```

### 📊 Module Statistics
- Total Sections: 13
- Code Lines: 2,500+
- Components Built: 12
- Technologies Used: 10+
- Best Practices Covered: 50+

### 🎯 What You've Learned
- Modern RAG architecture design
- Hybrid search implementation
- Open-source reranking (BGE)
- Vector store management
- Agent-based RAG systems
- Production deployment patterns
- Cost optimization strategies
- Security best practices

---

## ✨ End of Module 3 — Advanced RAG ✨
